In [1]:
import glob
import os
import re

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scienceplots
from natsort import natsorted

plt.style.use("science")
mpl.use("pgf")

In [2]:
plt.rcParams.update(
    {"text.usetex": False, "pgf.preamble": r"\usepackage{amssymb} \usepackage{amsmath}"}
)

In [3]:
def extract_matrix_data(file_path):
    with open(file_path, "r") as file:
        content = file.read().replace("\n", "").replace("\\", "")

    # Regular expression to capture the content inside Matrix(...) and the nested lists
    pattern = re.compile(r"Matrix\(\d+,\s*\d+,\s*(\[\[.*?\]\])", re.DOTALL)

    match = pattern.search(content)
    if match:
        matrix_data = match.group(1)
        matrix_data = np.array(eval(match.group(1)))
        return matrix_data
    else:
        raise ValueError("Matrix data not found in the file")

In [4]:
DATA = []
for name in ["ordinal", "fractional"]:
    TIMES1 = natsorted(glob.glob("runs-%s/times1-*" % name))
    TIMES2 = natsorted(glob.glob("runs-%s/times2-*" % name))
    names = np.array(list(map(lambda t: int(t[t.rindex("-") + 1 :]), TIMES1)))

    N = []
    for times1, times2 in zip(TIMES1, TIMES2):
        times1 = extract_matrix_data(times1)
        times2 = extract_matrix_data(times2)

        mean = times1.mean(axis=0)
        for_n_t1 = mean.sum()

        mean = times2.mean(axis=0)
        for_n_t2 = mean.sum()
        N.append((for_n_t1, for_n_t2))
    DATA.append(N)
DATA = np.array(DATA).astype(float)

DATA.shape

(2, 8, 2)

In [5]:
plt.figure(figsize=(4.5, 3))
plt.plot(names, DATA[0, :, 0], label="Discretization")
plt.plot(names, DATA[0, :, 1], label="Finding weights")

plt.plot(names, DATA[0, :, :].sum(axis=-1), label="Total")
plt.xlabel("\# of Time Discretizations")
plt.ylabel("Time (Sec.)")
plt.legend()

plt.savefig("cpu-time-ordinal.pdf", bbox_inches="tight", pad_inches=0)

In [6]:
plt.figure(figsize=(4.5, 3))
plt.plot(names, DATA[1, :, 0], label="Discretization")
plt.plot(names, DATA[1, :, 1], label="Finding weights")

plt.plot(names, DATA[1, :, :].sum(axis=-1), label="Total")
plt.xlabel("\# of Time Discretizations")
plt.ylabel("Time (Sec.)")
plt.legend()

plt.savefig("cpu-time-fractional.pdf", bbox_inches="tight", pad_inches=0)